## Interactive Value Curve Editor (bqplot Prototype)
This cell uses bqplot for a fully interactive, draggable curve editor for the Value (V) channel. Drag points to adjust, double-click to add/remove, and see the palette update in real time.

In [ ]:
# Interactive HSV Curve Editor using bqplot (C array export only, ESP-IDF format, H 0-255, clamped, compact spacing, theme presets, hue shift slider)

import numpy as np
import bqplot as bq
import ipywidgets as widgets
from matplotlib.colors import hsv_to_rgb
from IPython.display import display, clear_output, HTML
import pandas as pd

n_points = 256
curve_x = np.linspace(0, 1, n_points)
anchor_x = np.linspace(0, 1, 6).tolist()

# Theme presets
presets = {
    'Neutral Fire': {
        'hue': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        'sat': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
        'val': [0.3, 0.55, 0.7, 0.8, 0.75, 0.4],
    },
    'Toxic': {
        'hue': [0.25, 0.22, 0.20, 0.18, 0.18, 0.18],
        'sat': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
        'val': [0.1, 0.4, 0.8, 1.0, 0.7, 0.3],
    },
    'Lightning': {
        'hue': [0.60, 0.65, 0.68, 0.70, 0.75, 0.80],
        'sat': [1.0, 0.7, 0.2, 0.2, 0.7, 1.0],
        'val': [0.2, 0.7, 1.0, 1.0, 0.7, 0.2],
    },
    'Water': {
        'hue': [0.55, 0.60, 0.66, 0.70, 0.72, 0.75],
        'sat': [0.8, 0.9, 1.0, 1.0, 0.9, 0.8],
        'val': [0.2, 0.5, 0.9, 1.0, 0.7, 0.3],
    },
    'Aurora': {
        'hue': [0.6, 0.7, 0.33, 0.8, 0.7, 0.6],
        'sat': [0.8, 0.9, 1.0, 1.0, 0.9, 0.8],
        'val': [0.2, 0.5, 0.9, 1.0, 0.7, 0.3],
    },
}

# Initial values (default: Neutral Fire)
hue_y = presets['Neutral Fire']['hue'].copy()
sat_y = presets['Neutral Fire']['sat'].copy()
val_y = presets['Neutral Fire']['val'].copy()
hue_shift = 0.0

# Scales
x_sc = bq.LinearScale(min=0, max=1)
y_sc = bq.LinearScale(min=0, max=1)

# Scatter and lines for each channel
hue_scatter = bq.Scatter(x=anchor_x, y=hue_y, scales={'x': x_sc, 'y': y_sc}, enable_move=True, colors=['orange'], marker='circle', default_size=128)
hue_line = bq.Lines(x=anchor_x, y=hue_y, scales={'x': x_sc, 'y': y_sc}, colors=['orange'])
sat_scatter = bq.Scatter(x=anchor_x, y=sat_y, scales={'x': x_sc, 'y': y_sc}, enable_move=True, colors=['green'], marker='circle', default_size=128)
sat_line = bq.Lines(x=anchor_x, y=sat_y, scales={'x': x_sc, 'y': y_sc}, colors=['green'])
val_scatter = bq.Scatter(x=anchor_x, y=val_y, scales={'x': x_sc, 'y': y_sc}, enable_move=True, colors=['dodgerblue'], marker='circle', default_size=128)
val_line = bq.Lines(x=anchor_x, y=val_y, scales={'x': x_sc, 'y': y_sc}, colors=['dodgerblue'])

ax_x = bq.Axis(label='Position in Palette', scale=x_sc)
ax_y = bq.Axis(label='Channel Value', scale=y_sc, orientation='vertical')
fig = bq.Figure(
    marks=[hue_line, hue_scatter, sat_line, sat_scatter, val_line, val_scatter],
    axes=[ax_x, ax_y],
    title='HSV Curve Editor (Drag 6 points per channel, H 0-255, clamped)'
)
fig.layout.height = '400px'
palette_out = widgets.Output()

# Custom horizontal legend using HTML, centered
legend_html = '''
<div style="display: flex; justify-content: center; align-items: center; gap: 32px; font-size: 1.1em; margin-bottom: 8px;">
  <span style="display: flex; align-items: center;"><span style="display:inline-block;width:16px;height:16px;background:orange;border-radius:50%;margin-right:6px;border:1px solid #aaa;"></span>Hue</span>
  <span style="display: flex; align-items: center;"><span style="display:inline-block;width:16px;height:16px;background:green;border-radius:50%;margin-right:6px;border:1px solid #aaa;"></span>Saturation</span>
  <span style="display: flex; align-items: center;"><span style="display:inline-block;width:16px;height:16px;background:dodgerblue;border-radius:50%;margin-right:6px;border:1px solid #aaa;"></span>Value</span>
</div>
'''
legend_widget = widgets.HTML(value=legend_html)

# Theme selection checkboxes
preset_checkboxes = [widgets.Checkbox(value=(i==0), description=name) for i, name in enumerate(presets.keys())]

# Hue shift slider (vertical style)
hue_shift_slider = widgets.FloatSlider(
    value=0.0,
    min=-0.5,
    max=0.5,
    step=0.01,
    description='Hue Shift',
    orientation='vertical',
    layout=widgets.Layout(height='200px')
)

# Helper to interpolate curve
def get_curve(ax, ay):
    idx = np.argsort(ax)
    ax_sorted = np.array(ax)[idx]
    ay_sorted = np.array(ay)[idx]
    return np.interp(curve_x, ax_sorted, ay_sorted)

def apply_hue_shift(hue_curve, shift):
    shifted = (np.array(hue_curve) + shift)
    shifted = np.clip(shifted, 0, 1)
    return shifted

def update_curve(*args):
    global anchor_x, hue_y, sat_y, val_y, hue_shift
    anchor_x = list(hue_scatter.x)
    hue_y = list(hue_scatter.y)
    sat_y = list(sat_scatter.y)
    val_y = list(val_scatter.y)
    hue_line.x = anchor_x
    hue_line.y = hue_y
    sat_line.x = anchor_x
    sat_line.y = sat_y
    val_line.x = anchor_x
    val_line.y = val_y
    h_curve = get_curve(anchor_x, hue_y)
    h_curve = apply_hue_shift(h_curve, hue_shift_slider.value)
    s_curve = get_curve(anchor_x, sat_y)
    v_curve = get_curve(anchor_x, val_y)
    with palette_out:
        clear_output(wait=True)
        rgb = hsv_to_rgb(np.stack([h_curve, s_curve, v_curve], axis=1))
        import matplotlib.pyplot as plt
        plt.figure(figsize=(8,1))
        plt.imshow([rgb], aspect='auto')
        plt.axis('off')
        plt.title('Palette Preview')
        plt.show()

hue_scatter.observe(update_curve, names=['x', 'y'])
sat_scatter.observe(update_curve, names=['x', 'y'])
val_scatter.observe(update_curve, names=['x', 'y'])
hue_shift_slider.observe(update_curve, names=['value'])

# Theme preset handler
def on_preset_change(change):
    if change['new']:
        # Uncheck all others
        for cb in preset_checkboxes:
            if cb is not change['owner']:
                cb.value = False
        # Set curves to selected preset
        preset = change['owner'].description
        hue_scatter.y = presets[preset]['hue'].copy()
        sat_scatter.y = presets[preset]['sat'].copy()
        val_scatter.y = presets[preset]['val'].copy()
        update_curve()
for cb in preset_checkboxes:
    cb.observe(on_preset_change, names=['value'])

# Export function: ESP-IDF C array format (h,s,v, 0-255, 8-bit, H 0-255, clamped)
def export_palette(*args):
    h_curve = get_curve(anchor_x, hue_y)
    h_curve = apply_hue_shift(h_curve, hue_shift_slider.value)
    s_curve = get_curve(anchor_x, sat_y)
    v_curve = get_curve(anchor_x, val_y)
    h_8bit = np.clip(np.round(h_curve * 255), 0, 255).astype(int)
    s_8bit = np.clip(np.round(s_curve * 255), 0, 255).astype(int)
    v_8bit = np.clip(np.round(v_curve * 255), 0, 255).astype(int)
    lines = []
    for i in range(n_points):
        lines.append(f'{{{h_8bit[i]},{s_8bit[i]},{v_8bit[i]}}},')
    grouped = [''.join(lines[i:i+8]) for i in range(0, n_points, 8)]
    c_array = 'const hsv_color_t hsv_palette_fire[HSV_PALETTE_SIZE] = {\n' + '\n'.join(grouped) + '\n};'
    display(HTML(f'<b>C Array Preview:</b><br><pre>{c_array}</pre>'))

export_palette_button = widgets.Button(description='Export C Array')
export_palette_button.on_click(export_palette)

# Layout: legend above title, then graph, then controls and preview
legend_box = widgets.HBox([legend_widget], layout=widgets.Layout(justify_content='center'))
controls = widgets.HBox([
    widgets.VBox([
        widgets.Label('Theme Presets:'),
        widgets.VBox(preset_checkboxes)
    ], layout=widgets.Layout(align_items='flex-start', gap='8px')),
    widgets.VBox([
        widgets.Label('Hue Shift:'),
        hue_shift_slider
    ], layout=widgets.Layout(align_items='flex-start', gap='8px'))
], layout=widgets.Layout(gap='32px'))

bottom = widgets.HBox([
    palette_out,
    controls
], layout=widgets.Layout(align_items='flex-start', gap='32px'))

update_curve()
display(legend_box)
display(fig)
display(bottom)
display(export_palette_button)


Figure(axes=[Axis(label='Position in Palette', scale=LinearScale(max=1.0, min=0.0)), Axis(label='Channel Value…

Button(description='Export C Array', style=ButtonStyle())